In [1]:
# Impotando bibliotecas.
import time
import warnings
import requests
import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas_gbq import to_gbq

warnings.filterwarnings("ignore")

## Web Scraping SteamDB

In [2]:
# Parâmetros passados para o headers, problemas como 451.
request_headers = {
    'authority': 'steamdb.info',
    'method': 'GET',
    'path': '/sales/',
    'scheme': 'https',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Dnt': '1',
    'Referer': 'https://steamdb.info/sales/',
    'Sec-Ch-Ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': 'macOS',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}

# URL request ao STEAMDB.
URL = 'https://steamdb.info/sales/'

In [4]:
# Lista para armazenar infomações.
game_play = []

try:
    # Realizando raspagem no site.
    pagina = requests.get(URL, headers=request_headers)
    raspagem = BeautifulSoup(pagina.text, "html.parser")

    # Verificando resquest foi realizado com sucesso.
    if pagina.status_code == 200:
        # Buscando tabela dos Games no html.
        print('Status Code:', pagina.status_code)
        tabela = list(raspagem.find_all('tr'))

        # Percorendo e capturando informações da tabela de games.
        for x_game in range(1,len(tabela)):
            game_name = tabela[x_game].find_all('td')[2].a.text
            game_descont = tabela[x_game].find_all('td')[3].text
            game_price = tabela[x_game].find_all('td')[4].text
            game_rating = tabela[x_game].find_all('td')[5].text
            game_release = tabela[x_game].find_all('td')[6].text
            game_ends = datetime.fromtimestamp(int(tabela[x_game].find_all('td')[7]['data-sort']))
            game_started = datetime.fromtimestamp(int(tabela[x_game].find_all('td')[8]['data-sort']))

            # Adicionando informações a lista.
            game_play.append(dict(
                    NAME=game_name,
                    DESCONT=game_descont,
                    PRICE=game_price,
                    RATING=game_rating,
                    RELEASE=game_release,
                    ENDS=game_ends,
                    STARTED=game_started
            ))
    elif pagina.status_code != 200:
        print('ERRO na busca dos dados:', pagina.status_code)
        time.sleep(10)
except Exception as err:
    print('Erro na execução')

Status Code: 200


In [5]:
# Verificando primeiro registro da lista.
game_play[0]

{'NAME': 'Killing Floor',
 'DESCONT': '-90%',
 'PRICE': 'R$ 3,49',
 'RATING': '93.37%',
 'RELEASE': 'May 2009',
 'ENDS': datetime.datetime(2024, 5, 23, 14, 0),
 'STARTED': datetime.datetime(2024, 5, 16, 14, 1, 19)}

In [6]:
# Criando data frame para verificar dados e análises.
info_games = pd.DataFrame.from_dict(game_play)
info_games.head(15)

,NAME,DESCONT,PRICE,RATING,RELEASE,ENDS,STARTED
0,Killing Floor,-90%,"R$ 3,49",93.37%,May 2009,2024-05-23 14:00:00,2024-05-16 14:01:19
1,Dark Messiah of Might & Magic,-75%,"R$ 7,49",88.66%,Oct 2006,2024-05-31 14:00:00,2024-05-17 14:01:17
2,FlatOut 2™,-80%,"R$ 13,99",93.70%,Aug 2006,2024-05-23 14:00:00,2024-05-09 14:01:20
3,Xpand Rally,-89%,"R$ 1,86",77.47%,Aug 2006,2024-05-24 14:00:00,2024-05-10 14:01:18
4,Men of War: Red Tide,-85%,"R$ 2,54",80.82%,Nov 2009,2024-05-22 14:00:00,2024-05-15 14:01:19
5,Judge Dredd: Dredd vs. Death,-60%,"R$ 6,39",82.71%,Feb 2005,2024-05-20 14:00:00,2024-05-07 14:01:21
6,Evil Genius,-90%,"R$ 1,99",89.90%,Sep 2004,2024-05-20 14:00:00,2024-05-07 14:01:21
7,Aliens versus Predator Classic 2000,-80%,"R$ 3,39",84.08%,Mar 2000,2024-05-20 14:00:00,2024-05-07 14:01:21
8,Sid Meier's Civilization® IV,-85%,"R$ 4,49",89.14%,Oct 2005,2024-05-27 14:00:00,2024-05-13 14:21:19
9,FlatOut,-80%,"R$ 13,99",89.04%,Jul 2005,1969-12-31 21:00:00,2024-05-09 14:01:20


In [7]:
# Verificando quantidade de jogos raspados.
len(game_play)

816

In [8]:
info_games.shape

(816, 7)

In [9]:
# Gerando CVS para importa no Google BigQuery.
info_games.to_csv("info_games.csv", index=False)

## Google BigQuery

In [10]:
# Credenciais para conexão com GoogleBigquery.
credencial = service_account.Credentials.from_service_account_file(
    r'C:\Users\italo\Downloads\disco-idea-423616-n5-2e4ca0b0b00a.json',
    scopes = ['https://www.googleapis.com/auth/bigquery']
)

In [11]:
# Enviando dados para BigQuery
info_games.to_gbq(
    destination_table='disco-idea-423616-n5.steamdb.gameplay',
    project_id='disco-idea-423616-n5',
    if_exists='replace',
    credentials=credencial
)